
## Objective
Develop an application to intelligently process email order requests and customer inquiries for a Retail store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Email tone should be professional.


#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.


# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.5 MB/s eta 0:00:00


In [ ]:
#pip install langchain faiss-cpu
%pip install openai
#pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(

)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hi there! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = ''
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [ ]:
import pandas as pd
from IPython.display import display
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity

# Function to read Google Sheets data into a pandas DataFrame
def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

# Load products and emails data from the Google Spreadsheet
document_id = '' #spreadsheet document id
emails_df = read_data_frame(document_id, 'emails')

# Initialize the OpenAI client
client = OpenAI(
)

# Function to get embeddings from OpenAI for a given text
def get_embedding(text):
    embedding_response = client.embeddings.create(
        model="text-embedding-ada-002",  # A common model used for embeddings
        input=text
    )
    return embedding_response.data[0].embedding

# Predefined keywords for classification
order_keywords = ["order", "buy", "purchase", "request"]
inquiry_keywords = ["inquire", "details", "information"]

# Generate embeddings for keyword categories
order_embedding = get_embedding(" ".join(order_keywords))
inquiry_embedding = get_embedding(" ".join(inquiry_keywords))

# Function to classify emails using embeddings and cosine similarity
def classify_email(subject, message):
    email_text = f"{subject} {message}"

    # Generate embedding for the current email
    email_embedding = get_embedding(email_text)

    # Calculate similarity with order and inquiry embeddings
    order_similarity = cosine_similarity([email_embedding], [order_embedding])[0][0]
    inquiry_similarity = cosine_similarity([email_embedding], [inquiry_embedding])[0][0]

    # Classify based on which similarity score is higher
    if order_similarity > inquiry_similarity:
        return 'order request'
    elif inquiry_similarity > order_similarity:
        return 'product inquiry'
    else:
        return 'unknown'

# Apply the classification function to each email
emails_df['category'] = emails_df.apply(lambda row: classify_email(row['subject'], row['message']), axis=1)

# Output the email classification DataFrame with columns: email ID and category
email_classification = emails_df[['email_id', 'category']]
email_classification.to_csv('email-classification.csv', index=False)
# # Display the classified results
display(email_classification.head(10))  # Show the first 10 results for brevity


,email_id,category
0,E001,order request
1,E002,order request
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry
5,E006,order request
6,E007,order request
7,E008,order request
8,E009,product inquiry
9,E010,order request


# Task 2. Process order requests

In [ ]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

# Function to read Google Sheets data into a pandas DataFrame
def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

# Load products and emails data from the Google Spreadsheet
document_id = ''
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Initialize the OpenAI client
client = OpenAI(

)

# Function to get embeddings from OpenAI for a given list of texts
def get_embeddings(texts, client):
    embedding_response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=texts
    )
    return [embedding.embedding for embedding in embedding_response.data]

# Function to match product requests with available products using OpenAI embeddings
def get_matching_product(request_message, products_df, client):
    # Extract product names and descriptions from the products dataframe
    product_texts = products_df['name'] + " " + products_df['description']

    # Get embeddings for the product descriptions and the request message
    product_embeddings = get_embeddings(product_texts.tolist(), client)
    request_embedding = get_embeddings([request_message], client)[0]  # Get single embedding for request

    # Calculate cosine similarity between the request and all product descriptions
    similarities = cosine_similarity([request_embedding], product_embeddings)

    # Find the index of the most similar product
    most_similar_index = similarities.argmax()

    # Return the most similar product
    return products_df.iloc[most_similar_index]

# Function to extract the quantity from the email message
def extract_quantity(message):
    # Find all numbers in the message
    quantities = re.findall(r'\b\d+\b', message)
    if quantities:
        # Convert the first number found to an integer (assumes it's the quantity)
        return int(quantities[0])
    return 1  # Default quantity if no number is found

# Function to process orders based on email requests and product stock
def process_orders(products_df, emails_df, client):
    order_status = []

    # Iterate through each email
    for index, email in emails_df.iterrows():
        email_id = email['email_id']
        message = email['message']

        # Get the best matching product using OpenAI
        matching_product = get_matching_product(message, products_df, client)
        product_id = matching_product['product_id']
        product_name = matching_product['name']
        stock = matching_product['stock']

        # Extract the requested quantity from the email message
        requested_quantity = extract_quantity(message)

        # Check stock availability
        if stock >= requested_quantity:
            status = 'created'
            products_df.loc[products_df['product_id'] == product_id, 'stock'] -= requested_quantity
        else:
            status = 'out of stock'

        # Append the order result
        order_status.append({
            'email_id': email_id,
            'product_id': product_id,
            'quantity': requested_quantity,
            'status': status
        })

    # Convert the order status into a DataFrame
    order_status_df = pd.DataFrame(order_status)
    return order_status_df

# Process the orders and get the results
order_status_df = process_orders(products_df, emails_df, client)
order_status_df.to_csv('order_status_updated.csv', index=False)
# Display the result
from IPython.display import display
display(order_status_df)


,email_id,product_id,quantity,status
0,E001,LTH0976,1,created
1,E002,VBT2345,1,created
2,E003,LTH1098,1,created
3,E004,SFT1098,1,created
4,E005,CSH1098,22,out of stock
5,E006,CBT8901,1,created
6,E007,CLF2109,5,out of stock
7,E008,VSC6789,1,created
8,E009,CHN0987,1,created
9,E010,RSG8901,1,created


In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with your custom base URL and API key
client = OpenAI(
)

# Function to load data from Google Sheets
def read_data_frame(document_id, sheet_name=''):
    # Construct the link for CSV export
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv"
    if sheet_name:
        export_link += f"&sheet={sheet_name}"
    return pd.read_csv(export_link)

# Define a function to generate an email response using the OpenAI API
def generate_email_response(email_data):
    email_id = email_data['email_id']
    subject = email_data['subject']
    message = email_data['message']
    product_id = email_data['product_id']
    quantity = email_data['quantity']  # Directly use the quantity from the order data
    status = email_data['status']

    # Generate the response based on the order status and email content
    prompt = f"""
    You are a customer support agent for an fashion store. Write a polite and professional email response to the following customer email:

    Message: {message}

    The customer ordered product {product_id} with a quantity of {quantity}, and the current order status is {status}.

    If the status is 'created', confirm the order details and thank the customer for their order.
    If the status is 'out of stock', inform the customer about the unavailability, suggest any alternatives, or provide restock information if applicable. Be concise and professional.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

# Google Sheet IDs
order_status_document_id = ''  # Given sheet ID for order_status
email_document_id = ''  # Given sheet ID for emails

# Load order data from Google Sheets (order-status sheet)
order_data = read_data_frame(order_status_document_id, sheet_name='order-status')

# Load email data from the provided Google Sheets link (emails sheet)
email_data = read_data_frame(email_document_id, sheet_name='emails')

# Merge email data with order data based on email_id
merged_data = pd.merge(email_data, order_data, on="email_id", how="inner")

# Initialize a list to store the email_id and responses
responses = []

# Generate and collect email responses for each row in the merged data
for index, row in merged_data.iterrows():
    response = generate_email_response(row)
    responses.append({"email_id": row["email_id"], "response": response})

# Convert the list of responses into a DataFrame
response_df = pd.DataFrame(responses)

# Save the responses to a CSV file
response_df.to_csv('order-responses.csv', index=False)

# Display the result
from IPython.display import display
display(response_df)

,email_id,response
0,E001,Subject: Re: Inquiry about LTH0976 Leather Bif...
1,E002,Subject: Your Order for the VBT2345 Vibrant To...
2,E004,Subject: Confirmation of Your Order for SFT109...
3,E005,Subject: Regarding Your Inquiry About the CSH1...
4,E006,Subject: Confirmation of Your CBT8901 Chelsea ...
5,E007,Subject: Update on Your Order for Holiday Gift...
6,E008,Subject: Your Order Confirmation for the Versa...
7,E010,Subject: Your Order Confirmation - RSG8901 Ret...
8,E011,Subject: Re: Inquiry about RSG8901 Retro Sungl...
9,E012,Subject: Recommendations for a New Briefcase o...


# Task 3. Handle product inquiry

In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with your custom base URL and API key
client = OpenAI()

# Function to load data from Google Sheets or local CSV
def read_data_frame(document_id, sheet_name=''):
    # Construct the link for CSV export
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv"
    if sheet_name:
        export_link += f"&sheet={sheet_name}"
    return pd.read_csv(export_link)


# Define a function to generate a product inquiry response using the OpenAI API
def generate_product_inquiry_response(email_data):
    email_id = email_data['email_id']
    product_inquiry = email_data['message']  # Access the inquiry message from the merged dataset

    # Generate the response based on the product inquiry content
    prompt = f"""
    You are a customer support agent for a fashion store. A customer has inquired about the following product:

    Message: {product_inquiry}

    Provide a concise and professional response to the product inquiry, including any relevant product details, availability, and suggestions for alternatives if necessary.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    # Return the complete response message
    return response.choices[0].message.content.strip()


# Load the email data containing messages and product inquiries
classified_email_file = ''  #  spread sheet id
email_messages_file = ''  # spread sheet id

# Load the classified emails data (Excel) and email messages data (CSV)
classified_email_data = read_data_frame(order_status_document_id, sheet_name='email-classification')
email_messages_data = read_data_frame(email_document_id, sheet_name='emails')

# Step 1: Filter rows where the category is 'product inquiry'
product_inquiry_emails = classified_email_data[classified_email_data['category'] == 'product inquiry']

# Step 2: Merge the filtered 'product inquiry' emails with the email messages data based on 'email_id'
product_inquiry_with_messages = pd.merge(product_inquiry_emails, email_messages_data, on='email_id', how='inner')

# Initialize a list to store the email_id and responses
responses = []

# Step 3: Generate and collect responses for product inquiries
for index, row in product_inquiry_with_messages.iterrows():
    response = generate_product_inquiry_response(row)
    responses.append({"email_id": row["email_id"], "response": response})

# Convert the list of responses into a DataFrame
response_df = pd.DataFrame(responses)

# Step 4: Save the responses to a CSV file
response_file_path = 'inquiry-response.csv'
response_df.to_csv(response_file_path, index=False)

print(f"Responses have been saved to '{response_file_path}'.")

# Display the result
from IPython.display import display
display(response_df)

Responses have been saved to 'inquiry-response.csv'.


,email_id,response
0,E003,"Hello David,\n\n Thank you for reaching out..."
1,E005,"Dear [Customer's Name],\n\nThank you for reach..."
2,E009,"Claro, con gusto te ayudo.\n\nNuestro producto..."
3,E011,"Hi there,\n\nThank you for your inquiry about ..."
4,E012,"Hi [Customer's Name],\n\nI hope you're doing w..."
5,E015,"Good morning,\n\nThank you for reaching out an..."
6,E016,"Hello Claire,\n\nThank you for reaching out an..."
7,E020,"Hello Antonio,\n\nThank you for reaching out t..."
8,E021,"Dear [Customer's Name],\n\nThank you for your ..."
